# Square lattice $J_1$-$J_2$ XY and Heisenberg models

First, we should import all the tools we need for these models. As is known, a 4-site plaquette can be used to obtain the phase diagram of the $J_1$-$J_2$ Heisenberg model on the square lattice, so we don't need to define a larger cluster.

In [ ]:
from hmftpy.plaquettes.square import plaq4
from hmftpy.operators import mf_ops, inner_hamiltonian, periodic_hamiltonian
from hmftpy import do_hmft
from quspin.basis import spin_basis_1d
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

plaq = plaq4
L = 4
basis = spin_basis_1d(L, pauli=0)
maxit = 200 # maximum number of steps for HMFT
steps = 40 # Number of values of J_2

Let's define a function to return a dict telling my code which interactions to use. Here, I set $J_1=1$. When $\alpha=1$, we will have a Heisenberg model Hamiltonian, while $\alpha =0$ gives the XY model:
\begin{equation}
H = J_1\sum_{\langle i, j \rangle} \left(\vec\sigma_i \cdot \vec\sigma_j
+ (\alpha-1) \sigma_i^z \sigma_j^z\right)
+ J_2\sum_{\langle\langle i, j \rangle\rangle}\left(\vec\sigma_i \cdot \vec\sigma_j
+ (\alpha-1) \sigma_i^z \sigma_j^z\right)
\end{equation}

In [ ]:
def couplings(J2, alpha=1):
    interactions = {'n_bonds': {'xx': 1, 'yy': 1, 'zz': alpha},
                    'nn_bonds': {'xx': J2, 'yy': J2, 'zz': J2*alpha}}
    return interactions

## Observables

We can calculate order parameters for the two ordered phases. For low $J_2$, nearest-neighbor Néel order dominates, with order parameter
\begin{equation}
M_\text{stag}^\mu = \frac{1}{L}\sum_i (-1)^{i}\langle\sigma_i^\mu\rangle
\end{equation}
where sites are numbered such that an even-indexed site has only odd-indexed nearest neighbors.

In the high-$J_2$ phase, there is Néel ordering along next-nearest neighbors, with spins aligned along either rows ($x$ direction) or columns ($y$ direction) in the lattice. In the four-site cluster, we can write these as 
\begin{equation}
M_{\text{col},x}^\mu = \frac{1}{4}\langle \sigma_0^\mu + \sigma_1^\mu - \sigma_2^\mu - \sigma_3^\mu\rangle
\end{equation}
and
\begin{equation}
M_{\text{col},y}^\mu = \frac{1}{4}\langle \sigma_0^\mu - \sigma_1^\mu - \sigma_2^\mu + \sigma_3^\mu\rangle
\end{equation}
where the cluster is indexed 

    3 - 2
    |   |
    0 - 1

Since I've already measured all the components of local spins, I don't need to build an operator to measure these. Instead, I can use the prerecorded mean-field values.

For the Heisenberg model, it is typical to choose $\mu = z$. When we move to the XY model, we will need to choose something in the $x$-$y$ plane because the $z$ components of spin are free.

In [ ]:
def stag_mag(mf, mu='z'):
    return np.real(np.sum([(-1)**i*mf[mu][i] for i in range(L)]))/4
                   
def col_mag_x(mf, mu='z'):
    return np.real(np.sum([(-1)**(i//2)*mf[mu][i] for i in range(L)]))/4
                   
def col_mag_y(mf, mu='z'):
    return np.real(np.sum([(-1)**((i+1)//2)*mf[mu][i] for i in range(L)]))/4

## Finding HMFT solutions

Now, I create some arrays to record energy, mean-field values, and convergence for my solutions, along with a list of the values of $J_2$ I want to solve for.

In [ ]:
J2s = np.linspace(0, 1, steps)

energies_lr = np.zeros(steps)
cvgs_lr = [False for i in range(steps)]
mfs_lr = [None for i in range(steps)]

energies_rl = np.zeros(steps)
cvgs_rl = [False for i in range(steps)]
mfs_rl = [None for i in range(steps)]

energies_o = np.zeros(steps)

To ensure I stick to one solution w/r/t/ magnetization directions and such, I will reuse mean-fields from HMFT solutions at previous couplings while increasing $J_2$ from 0 to 1. While I'm at it, I can also diagonalize the Hamiltonian of a single cluster to get OBC energy. This should be the same as the HMFT energy in the intermediate phase, since in this phase the clusters decouple.

In [ ]:
mf0 = None # For the first step, this ensures I use a random mean-field
for i, J2 in enumerate(tqdm(J2s, ascii=True)):
    interactions = couplings(J2)
    energies_lr[i], v, mf0, cvgs_lr[i] = do_hmft(plaq, interactions, basis, 
                                                 max_iter=maxit, mf0=mf0, rescale_e=True, 
                                                 hmft_tol=10**-10)
    mfs_lr[i] = mf0
    
    ho = inner_hamiltonian(plaq4, interactions, basis)
    eo, vo = ho.eigsh(k=1, which='SA')
    energies_o[i] = eo[0]

The HMFT solution tends to stick to a particular phase as a false minimum around first-order transitions, so to capture the transition at $J_2\approx 0.67$, I will need to perform another run going from $J_2=1$ to $0$.

In [ ]:
mf0 = None
for i, J2 in enumerate(tqdm(J2s[::-1], ascii=True)):
    j = steps-1-i # reindexing so the results are still are ordered left to right
    interactions = couplings(J2)
    energies_rl[j], v, mf0, cvgs_rl[j] = do_hmft(plaq, interactions, basis, 
                                                 max_iter=maxit, mf0=mf0, rescale_e=True,
                                                 hmft_tol=10**-10)
    mfs_rl[j] = mf0

## Results (Heisenberg model)

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.plot(J2s, energies_lr/4, label='Left to right', marker='+')
plt.plot(J2s, energies_rl/4, label='Right to left', marker='x')
plt.plot(J2s, energies_o/4, label='OBC', zorder=10, ls='--', color='black')
plt.xlabel('$J_2$')
plt.ylabel('$e$')
plt.legend()

When plotting the order parameters, let's just use the lowest-energy (presumably 'correct') HMFT solution.

In [ ]:
ind = np.argmin(np.abs(J2s-0.6)) # Both solutions agree here (and in the neighborhood of here)
best_mfs = mfs_lr[:ind] + mfs_rl[ind:]

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(J2s, [stag_mag(mf, 'z') for mf in best_mfs], label=r'$M_{stag}^z$', marker='+', s=20)
plt.scatter(J2s, [col_mag_x(mf, 'z') for mf in best_mfs], label=r'$M_{col, x}^z$', marker='x', s=20)
plt.scatter(J2s, [col_mag_y(mf, 'z') for mf in best_mfs], label=r'$M_{col, y}^z$', marker='.', s=20)
plt.legend()
plt.xlabel('$J_2$')

When comparing to literature on the subject, the magnitude of these order parameters seems a bit too small. This could be because the alignment isn't precisely along the $\sigma^z$ direction. How about we compute the total magnitude (i.e. $M_\text{stag} = \sqrt{\sum_\mu (M_\text{stag}^\mu)^2}$)

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(J2s, [np.sqrt(stag_mag(mf, 'x')**2 + stag_mag(mf, 'y')**2 + stag_mag(mf, 'z')**2) for mf in best_mfs], 
            label=r'$M_{stag}$', marker='+', s=20)
plt.scatter(J2s, [np.sqrt(col_mag_x(mf, 'x')**2 + col_mag_x(mf, 'y')**2 + col_mag_x(mf, 'z')**2) for mf in best_mfs], 
            label=r'$M_{col, x}$', marker='2', s=20)
plt.scatter(J2s, [np.sqrt(col_mag_y(mf, 'x')**2 + col_mag_y(mf, 'y')**2 + col_mag_y(mf, 'z')**2) for mf in best_mfs], 
            label=r'$M_{col, y}$', marker='1', s=20)
plt.legend()
plt.xlabel('$J_2$')

This is much closer to what is in Gerardo's paper.

# XY Model

Now, let's repeat everything we did for the $XY$ model. The intermediate phase in the XY model is much smaller, so let's work with a smaller range of $J_2$ around the transition point(s):

In [ ]:
J2s_xy = np.linspace(0.45, 0.65, steps)
energies_lr_xy = np.zeros(steps)
cvgs_lr_xy = [False for i in range(steps)]
mfs_lr_xy = [None for i in range(steps)]

energies_rl_xy = np.zeros(steps)
cvgs_rl_xy = [False for i in range(steps)]
mfs_rl_xy = [None for i in range(steps)]

energies_o_xy = np.zeros(steps)

In [ ]:
mf0 = None # For the first step, this ensures I use a random mean-field
for i, J2 in enumerate(tqdm(J2s_xy, ascii=True)):
    interactions = couplings(J2, alpha=0)
    energies_lr_xy[i], v, mf0, cvgs_lr_xy[i] = do_hmft(plaq, interactions, basis, 
                                                 max_iter=maxit, mf0=mf0, rescale_e=True, 
                                                 hmft_tol=10**-10)
    mfs_lr_xy[i] = mf0
    
    ho = inner_hamiltonian(plaq4, interactions, basis)
    eo, vo = ho.eigsh(k=1, which='SA')
    energies_o_xy[i] = eo[0]

In [ ]:
mf0 = None
for i, J2 in enumerate(tqdm(J2s_xy[::-1], ascii=True)):
    j = steps-1-i # reindexing so the results are still are ordered left to right
    interactions = couplings(J2, alpha=0)
    energies_rl_xy[j], v, mf0, cvgs_rl_xy[j] = do_hmft(plaq, interactions, basis, 
                                                       max_iter=maxit, mf0=mf0, rescale_e=True,
                                                       hmft_tol=10**-10)
    mfs_rl_xy[j] = mf0

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.plot(J2s_xy, energies_lr_xy/4, label='Left to right', marker='+')
plt.plot(J2s_xy, energies_rl_xy/4, label='Right to left', marker='x')
plt.plot(J2s_xy, energies_o_xy/4, label='OBC', zorder=10, ls='--', color='black')
plt.xlabel('$J_2$')
plt.ylabel('$e$')
plt.legend()

From this plot, it seems pretty clear that (at least for 4 sites) there is no intermediate phase in the XY model, just a first-order transition between the two ordered phases at $J_2\approx 0.54$.

Let's use a different strategy to join the solutions now. How about we just go with the lowest-energy solution, but give preference to the left-right one when they are close (so we don't get anything jumpy in the low-$J_2$ phase)

In [ ]:
best_mfs_xy = [None for i in range(steps)]
for i in range(steps):
    if energies_rl_xy[i] - energies_lr_xy[i] < 10**-8:
        best_mfs_xy[i] = mfs_rl_xy[i]
    else:
        best_mfs_xy[i] = mfs_lr_xy[i]

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(J2s, [np.sqrt(stag_mag(mf, 'x')**2 + stag_mag(mf, 'y')**2 + stag_mag(mf, 'z')**2) for mf in best_mfs_xy], 
            label=r'$M_{stag}$', marker='+', s=20)
plt.scatter(J2s, [np.sqrt(col_mag_x(mf, 'x')**2 + col_mag_x(mf, 'y')**2 + col_mag_x(mf, 'z')**2) for mf in best_mfs_xy], 
            label=r'$M_{col, x}$', marker='2', s=20)
plt.scatter(J2s, [np.sqrt(col_mag_y(mf, 'x')**2 + col_mag_y(mf, 'y')**2 + col_mag_y(mf, 'z')**2) for mf in best_mfs_xy], 
            label=r'$M_{col, y}$', marker='1', s=20)
plt.legend()
plt.xlabel('$J_2$')

Just for completeness' sake, let's plot the energy as a function of interaction range $\alpha$ for a power-law ($r^{-\alpha}$) interaction. Since the next-nearest neighbors are $\sqrt 2$ apart (where we set lattice spacings to 1), this is related to $J_2$ as $J_2 = 2^{-\alpha/2}
\implies \alpha = -2 \frac{\ln(J_2)}{\ln(2)}$.

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.plot(-2*np.log(J2s_xy)/np.log(2), energies_lr_xy/4, label='Left to right', marker='+')
plt.plot(-2*np.log(J2s_xy)/np.log(2), energies_rl_xy/4, label='Right to left', marker='x')
plt.plot(-2*np.log(J2s_xy)/np.log(2), energies_o_xy/4, label='OBC', zorder=10, ls='--', color='black')
plt.xlabel(r'$\alpha$')
plt.ylabel('$e$')
plt.legend()